In [ ]:
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout
)
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# ========== CONFIG ==========
DATA_DIR = "../data/training_data"
MODEL_DIR = "../models"

MOODS = ["happy", "sad", "angry", "quiet"]
SEQ_LEN = 50
EMBED_DIM = 128
LSTM_UNITS = 256
BATCH_SIZE = 256
EPOCHS = 5
# ===========================

In [ ]:
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
with open(os.path.join(DATA_DIR, "X.pkl"), "rb") as f:
    X = pickle.load(f)

with open(os.path.join(DATA_DIR, "mood_labels.pkl"), "rb") as f:
    y_mood = pickle.load(f)

print("X shape:", X.shape)
print("Mood labels shape:", y_mood.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_mood,
    test_size=0.2,
    random_state=42,
    stratify=y_mood
)

print("Train size:", X_train.shape[0])
print("Validation size:", X_val.shape[0])

In [ ]:
vocab_size = int(np.max(X)) + 1
num_classes = len(MOODS)

inputs = Input(shape=(SEQ_LEN,))
x = Embedding(vocab_size, EMBED_DIM)(inputs)
x = LSTM(LSTM_UNITS)(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

In [ ]:
y_pred = np.argmax(model.predict(X_val), axis=1)

print(classification_report(y_val, y_pred, target_names=MOODS))

In [ ]:
cm = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=MOODS,
    yticklabels=MOODS
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Mood Classification Confusion Matrix")
plt.show()

In [ ]:
model.save(os.path.join(MODEL_DIR, "mood_classifier.h5"))
print("Mood classifier saved ")